In [16]:
import pandas as pd
import akshare as ak
import os
import time
import hashlib
import numpy as np

# ==========================
# 只获取累计净值（保留原逻辑）
# ==========================
def cached_fund_nav_acc(symbol):
    _CACHE_DIR = "./ak_cache"
    os.makedirs(_CACHE_DIR, exist_ok=True)
    key = f"fund_acc_nav_{symbol}"
    h = hashlib.md5(key.encode()).hexdigest()
    cache_file = os.path.join(_CACHE_DIR, f"{h}.pkl")

    if os.path.exists(cache_file):
        return pd.read_pickle(cache_file)

    # 只调用累计净值走势
    df = ak.fund_open_fund_info_em(symbol=symbol, indicator="累计净值走势")

    df["净值日期"] = pd.to_datetime(df["净值日期"])
    df["累计净值"] = pd.to_numeric(df["累计净值"], errors="coerce")
    df = df.dropna(subset=["累计净值"]).sort_values("净值日期").reset_index(drop=True)

    # 缓存
    pd.to_pickle(df, cache_file + ".tmp")
    os.replace(cache_file + ".tmp", cache_file)
    time.sleep(0.3)
    return df

# ==========================
# 回测：近 N 年收益率分位数（增加精准分位数字段）
# ==========================
def backtest_Nyear_return_quantile(fund_code, N=3, year_days=250):
    # 1. 获取数据
    df = cached_fund_nav_acc(fund_code)
    if len(df) == 0:
        return {"error": "无基金数据"}
    
    # 2. 校验基础数据量
    n_days = N * year_days  # N年所需交易日数
    if len(df) < n_days:
        return {"error": f"数据不足：当前有{len(df)}天，需至少{n_days}天（{N}年×{year_days}天/年）"}
    
    # 3. 计算滚动N年收益率（核心指标）
    df[f"滚动{N}年收益率(%)"] = df["累计净值"].pct_change(n_days) * 100
    df_valid = df.dropna(subset=[f"滚动{N}年收益率(%)"]).copy().reset_index(drop=True)
    print(f"【调试】有效滚动{N}年收益率数据条数：{len(df_valid)}")  # 调试用
    
    # 4. 计算当前收益率和分位数
    current_return = df_valid[f"滚动{N}年收益率(%)"].iloc[-1]
    quantile = (df_valid[f"滚动{N}年收益率(%)"] < current_return).mean() * 100
    
    # 5. 分五档分位（低/中/高） + 计算精准分位数
    ret_series = df_valid[f"滚动{N}年收益率(%)"]
    # 新增：计算每一行的精准分位数（0-100的具体数值）
    df_valid["收益率分位数(%)"] = ret_series.rank(pct=True) * 100
    df_valid["收益率分位数(%)"] = df_valid["收益率分位数(%)"].round(2)  # 保留2位小数
    
    # 原有区间分位标签
    q20 = np.percentile(ret_series, 20)
    q40 = np.percentile(ret_series, 40)
    q60 = np.percentile(ret_series, 60)
    q80 = np.percentile(ret_series, 80)
    df_valid["分位标签"] = pd.cut(
        ret_series,
        bins=[-np.inf, q20, q40,q60,q80, np.inf],
        labels=["收益分位(0-20%)", "收益分位(20-40%)", "收益分位(40-60%)","收益分位(60-80%)","收益分位(80-100%)"]
    )
    print(f"【调试】各分位样本数：\n{df_valid['分位标签'].value_counts()}")  # 调试用
    
    # 6. 回测核心：记录每一笔交易明细 + 汇总统计
    backtest = {}
    # 新增：用于存储所有交易明细的列表
    trade_details_list = []
    # 先把日期设为索引，方便按日期查找
    df_valid_indexed = df_valid.set_index("净值日期")
    
    for label in df_valid["分位标签"].unique():
        # 筛选该分位的所有买入行
        buy_df = df_valid[df_valid["分位标签"] == label].copy()
        rets = []
        
        for idx, buy_row in buy_df.iterrows():
            # 买入日期和净值
            buy_date = buy_row["净值日期"]
            buy_nav = buy_row["累计净值"]
            # 新增：获取该笔交易的精准收益率分位数
            buy_quantile = buy_row["收益率分位数(%)"]
            
            # 目标卖出日期：买入日期 + 250个交易日（不是自然日）
            # 找到所有在买入日期之后的行
            after_buy = df_valid_indexed[df_valid_indexed.index > buy_date]
            if len(after_buy) < year_days:
                continue  # 后续数据不足250个交易日，跳过
            # 取第250个交易日作为卖出点
            sell_idx = after_buy.index[year_days - 1]  # 索引从0开始
            sell_nav = df_valid_indexed.loc[sell_idx, "累计净值"]
            # 计算持有1年收益
            hold_return = (sell_nav / buy_nav - 1) * 100
            rets.append(hold_return)
            
            # 新增：记录该笔交易的详细信息（包含精准分位数）
            trade_details_list.append({
                "基金代码": fund_code,
                "分位标签": label,
                "收益率分位数(%)": buy_quantile,  # 新增：精准分位数
                "买入日期": buy_date,
                "买入净值": round(buy_nav, 4),
                "卖出日期": sell_idx,
                "卖出净值": round(sell_nav, 4),
                "持有1年收益(%)": round(hold_return, 2),
                "是否盈利": "是" if hold_return > 0 else "否"
            })
        
        # 只有有有效收益数据才记录汇总信息
        if rets:
            backtest[label] = {
                "买入次数": len(rets),
                "持有1年平均收益(%)": round(np.mean(rets), 2),
                "持有1年盈利概率(%)": round((np.array(rets) > 0).mean() * 100, 2)
            }
        else:
            backtest[label] = {"提示": "无足够后续数据进行回测"}
    
    # 7. 将交易明细列表转换为DataFrame
    trade_details_df = pd.DataFrame(trade_details_list)
    # 按买入日期排序
    if not trade_details_df.empty:
        trade_details_df = trade_details_df.sort_values("买入日期").reset_index(drop=True)
    
    # 8. 整理最终结果（新增交易明细DataFrame）
    return {
        "基金代码": fund_code,
        f"近{N}年收益率(%)": round(current_return, 2),
        "历史分位数(%)": round(quantile, 2),
        "有效回测样本总数": len(df_valid),
        "回测结果（买入持有1年）": backtest,
        "交易明细DataFrame": trade_details_df  # 新增：交易明细数据框
    }

# ==========================
# 运行回测
# ==========================
if __name__ == "__main__":
    # 测试参数
    fund_code='110020'
    return_year=3
    result = backtest_Nyear_return_quantile(fund_code, N=return_year)
    
    # 检查是否有错误
    if "error" in result:
        print(f"错误：{result['error']}")
    else:
        return_key = f"近{return_year}年收益率(%)"
        # 格式化输出汇总结果
        print("="*80)
        print(f"基金代码：{result['基金代码']}")
        print(f"近{return_year}年收益率：{result[return_key]}%")
        print(f"历史分位数：{result['历史分位数(%)']}%")
        print(f"有效回测样本总数：{result['有效回测样本总数']}")
        print("-"*80)
        print("回测结果（买入持有1年）：")
        for quantile_label, res in result["回测结果（买入持有1年）"].items():
            print(f"\n{quantile_label}：")
            for k, v in res.items():
                print(f"  {k}：{v}")
        
        # 输出交易明细DataFrame
        print("\n" + "="*80)
        print("交易明细（前10行）：")
        trade_df = result["交易明细DataFrame"]
        if not trade_df.empty:
            print(trade_df.head(10))  # 显示前10行
            # 保存到Excel文件
            trade_df.to_excel(f"{fund_code}_近{return_year}年回测_持有1年.xlsx", index=False)
            print(f"\n交易明细已保存至：{fund_code}_近{return_year}年回测_持有1年.xlsx")
        else:
            print("暂无有效交易明细数据")

【调试】有效滚动3年收益率数据条数：3255
【调试】各分位样本数：
收益分位(0-20%)      651
收益分位(20-40%)     651
收益分位(40-60%)     651
收益分位(60-80%)     651
收益分位(80-100%)    651
Name: 分位标签, dtype: int64
基金代码：110020
近3年收益率：22.32%
历史分位数：53.46%
有效回测样本总数：3255
--------------------------------------------------------------------------------
回测结果（买入持有1年）：

收益分位(0-20%)：
  买入次数：651
  持有1年平均收益(%)：29.17
  持有1年盈利概率(%)：84.64

收益分位(20-40%)：
  买入次数：526
  持有1年平均收益(%)：10.41
  持有1年盈利概率(%)：65.78

收益分位(40-60%)：
  买入次数：591
  持有1年平均收益(%)：11.03
  持有1年盈利概率(%)：60.58

收益分位(60-80%)：
  买入次数：586
  持有1年平均收益(%)：3.13
  持有1年盈利概率(%)：59.39

收益分位(80-100%)：
  买入次数：651
  持有1年平均收益(%)：-6.91
  持有1年盈利概率(%)：26.88

交易明细（前10行）：
     基金代码         分位标签  收益率分位数(%)       买入日期    买入净值       卖出日期    卖出净值  \
0  110020  收益分位(0-20%)       7.13 2012-10-10  0.7315 2013-10-25  0.7552   
1  110020  收益分位(0-20%)       6.33 2012-10-11  0.7250 2013-10-28  0.7543   
2  110020  收益分位(0-20%)       5.53 2012-10-12  0.7256 2013-10-29  0.7561   
3  110020  收益分位(0-20%)       3.56 2012-10-15 

In [23]:
import pandas as pd
import akshare as ak
import os
import time
import hashlib
import numpy as np

# ==========================
# 只获取累计净值（保留原逻辑）
# ==========================
def cached_fund_nav_acc(symbol):
    _CACHE_DIR = "./ak_cache"
    os.makedirs(_CACHE_DIR, exist_ok=True)
    key = f"fund_acc_nav_{symbol}"
    h = hashlib.md5(key.encode()).hexdigest()
    cache_file = os.path.join(_CACHE_DIR, f"{h}.pkl")

    if os.path.exists(cache_file):
        return pd.read_pickle(cache_file)

    # 只调用累计净值走势
    df = ak.fund_open_fund_info_em(symbol=symbol, indicator="累计净值走势")

    df["净值日期"] = pd.to_datetime(df["净值日期"])
    df["累计净值"] = pd.to_numeric(df["累计净值"], errors="coerce")
    df = df.dropna(subset=["累计净值"]).sort_values("净值日期").reset_index(drop=True)

    # 缓存
    pd.to_pickle(df, cache_file + ".tmp")
    os.replace(cache_file + ".tmp", cache_file)
    time.sleep(0.3)
    return df

# ==========================
# 回测：多维度分位共振买入（近N1年+近N2年同区间才买入）
# ==========================
def backtest_multi_quantile_resonance(
    fund_code, 
    N1=1,  # 第一个时间维度（如1年）
    N2=3,  # 第二个时间维度（如3年）
    year_days=250
):
    # 1. 获取数据
    df = cached_fund_nav_acc(fund_code)
    if len(df) == 0:
        return {"error": "无基金数据"}
    
    # 2. 校验基础数据量（取最大的时间维度作为最低要求）
    max_n = max(N1, N2)
    min_days = max_n * year_days
    if len(df) < min_days:
        return {"error": f"数据不足：当前有{len(df)}天，需至少{min_days}天（{max_n}年×{year_days}天/年）"}
    
    # 3. 计算两个时间维度的滚动收益率
    # 3.1 计算近N1年滚动收益率
    df[f"滚动{N1}年收益率(%)"] = df["累计净值"].pct_change(N1 * year_days) * 100
    # 3.2 计算近N2年滚动收益率
    df[f"滚动{N2}年收益率(%)"] = df["累计净值"].pct_change(N2 * year_days) * 100
    
    # 4. 筛选同时有两个维度收益率的有效数据
    df_valid = df.dropna(subset=[f"滚动{N1}年收益率(%)", f"滚动{N2}年收益率(%)"]).copy().reset_index(drop=True)
    print(f"【调试】同时有近{N1}年和近{N2}年收益率的有效数据条数：{len(df_valid)}")
    
       # 5. 分别计算每个维度的分位点和分位标签（核心修正）
    ret_series1 = df_valid[f"滚动{N1}年收益率(%)"]  # 近N1年收益率序列
    ret_series2 = df_valid[f"滚动{N2}年收益率(%)"]  # 近N2年收益率序列

    # 5.1 近N1年维度：单独计算自身的分位点和分位标签
    # 计算N1年自身的分位点
    q20_1 = np.percentile(ret_series1, 20)
    q40_1 = np.percentile(ret_series1, 40)
    q60_1 = np.percentile(ret_series1, 60)
    q80_1 = np.percentile(ret_series1, 80)
    # 精准分位数（自身排名）
    df_valid[f"{N1}年收益率分位数(%)"] = ret_series1.rank(pct=True) * 100
    df_valid[f"{N1}年收益率分位数(%)"] = df_valid[f"{N1}年收益率分位数(%)"].round(2)
    # 分位标签（基于自身分位点）
    df_valid[f"{N1}年分位标签"] = pd.cut(
        ret_series1,
        bins=[-np.inf, q20_1, q40_1, q60_1, q80_1, np.inf],
        labels=["收益分位(0-20%)", "收益分位(20-40%)", "收益分位(40-60%)", "收益分位(60-80%)", "收益分位(80-100%)"]
    )

    # 5.2 近N2年维度：单独计算自身的分位点和分位标签
    # 计算N2年自身的分位点
    q20_2 = np.percentile(ret_series2, 20)
    q40_2 = np.percentile(ret_series2, 40)
    q60_2 = np.percentile(ret_series2, 60)
    q80_2 = np.percentile(ret_series2, 80)
    # 精准分位数（自身排名）
    df_valid[f"{N2}年收益率分位数(%)"] = ret_series2.rank(pct=True) * 100
    df_valid[f"{N2}年收益率分位数(%)"] = df_valid[f"{N2}年收益率分位数(%)"].round(2)
    # 分位标签（基于自身分位点）
    df_valid[f"{N2}年分位标签"] = pd.cut(
        ret_series2,
        bins=[-np.inf, q20_2, q40_2, q60_2, q80_2, np.inf],
        labels=["收益分位(0-20%)", "收益分位(20-40%)", "收益分位(40-60%)", "收益分位(60-80%)", "收益分位(80-100%)"]
    )
    
    # 6. 核心逻辑：筛选两个维度分位标签相同的日期作为买入点
    df_valid["分位共振"] = df_valid[f"{N1}年分位标签"] == df_valid[f"{N2}年分位标签"]
    buy_candidates = df_valid[df_valid["分位共振"] == True].copy()
    print(f"【调试】两个维度分位共振的买入候选数：{len(buy_candidates)}")
    print(f"【调试】各共振区间分布：\n{buy_candidates[f'{N1}年分位标签'].value_counts()}")
    
    # 7. 回测：遍历买入候选，计算持有1年收益
    backtest = {}
    trade_details_list = []
    df_valid_indexed = df_valid.set_index("净值日期")
    
    # 遍历所有共振分位区间
    for label in buy_candidates[f"{N1}年分位标签"].unique():
        buy_df = buy_candidates[buy_candidates[f"{N1}年分位标签"] == label].copy()
        rets = []
        
        for idx, buy_row in buy_df.iterrows():
            buy_date = buy_row["净值日期"]
            buy_nav = buy_row["累计净值"]
            
            # 检查后续是否有足够的250个交易日作为卖出点
            after_buy = df_valid_indexed[df_valid_indexed.index > buy_date]
            if len(after_buy) < year_days:
                continue
            
            # 计算卖出点和持有收益
            sell_idx = after_buy.index[year_days - 1]
            sell_nav = df_valid_indexed.loc[sell_idx, "累计净值"]
            hold_return = (sell_nav / buy_nav - 1) * 100
            rets.append(hold_return)
            
            # 记录交易明细（包含两个维度的分位信息）
            trade_details_list.append({
                "基金代码": fund_code,
                "共振分位区间": label,
                f"{N1}年收益率分位数(%)": buy_row[f"{N1}年收益率分位数(%)"],
                f"{N2}年收益率分位数(%)": buy_row[f"{N2}年收益率分位数(%)"],
                "买入日期": buy_date,
                "买入净值": round(buy_nav, 4),
                "卖出日期": sell_idx,
                "卖出净值": round(sell_nav, 4),
                "持有1年收益(%)": round(hold_return, 2),
                "是否盈利": "是" if hold_return > 0 else "否"
            })
        
        # 记录该区间的汇总信息
        if rets:
            backtest[label] = {
                "买入次数": len(rets),
                "持有1年平均收益(%)": round(np.mean(rets), 2),
                "持有1年收益0分位数":round(np.percentile(rets, 0),2),
                "持有1年收益25分位数":round(np.percentile(rets, 25),2),
                "持有1年收益50分位数":round(np.percentile(rets, 50),2),
                "持有1年收益75分位数":round(np.percentile(rets, 75),2),
                "持有1年收益100分位数":round(np.percentile(rets, 100),2),
                "持有1年盈利概率(%)": round((np.array(rets) > 0).mean() * 100, 2),
                "持有1年收益标准差(%)": round(np.std(rets), 2)  # 新增：收益波动情况
            }
        else:
            backtest[label] = {"提示": "无足够后续数据进行回测"}
    
    # 8. 转换交易明细为DataFrame
    trade_details_df = pd.DataFrame(trade_details_list)
    if not trade_details_df.empty:
        trade_details_df = trade_details_df.sort_values("买入日期").reset_index(drop=True)
    
    # 9. 计算当前最新的分位共振情况
    current_row = df_valid.iloc[-1]
    current_resonance = current_row["分位共振"]
    current_info = {
        f"当前近{N1}年收益率(%)": round(current_row[f"滚动{N1}年收益率(%)"], 2),
        f"当前近{N1}年收益率分位数(%)": round(current_row[f"{N1}年收益率分位数(%)"], 2),
        f"当前近{N2}年收益率(%)": round(current_row[f"滚动{N2}年收益率(%)"], 2),
        f"当前近{N2}年收益率分位数(%)": round(current_row[f"{N2}年收益率分位数(%)"], 2),
        f"当前近{N1}年分位": current_row[f"{N1}年分位标签"],
        f"当前近{N2}年分位": current_row[f"{N2}年分位标签"],
        "当前是否分位共振": "是" if current_resonance else "否"
    }
    
    # 10. 整理最终结果
    return {
        "基金代码": fund_code,
        "有效数据总数": len(df_valid),
        "共振买入候选数": len(buy_candidates),
        "当前分位共振信息": current_info,
        "回测结果（买入持有1年）": backtest,
        "交易明细DataFrame": trade_details_df
    }

# ==========================
# 运行回测
# ==========================
if __name__ == "__main__":
    # 自定义参数：近1年 + 近3年分位共振买入
    fund_code = '110020'
    N1 = 1  # 第一个时间维度（1年）
    N2 = 3  # 第二个时间维度（3年）
    
    result = backtest_multi_quantile_resonance(fund_code, N1=N1, N2=N2)
    
    # 处理错误
    if "error" in result:
        print(f"错误：{result['error']}")
    else:
        # 输出汇总信息
        print("="*80)
        print(f"基金代码：{result['基金代码']}")
        print(f"有效数据总数：{result['有效数据总数']}")
        print(f"共振买入候选数：{result['共振买入候选数']}")
        print("-"*80)
        print("当前分位共振信息：")
        for k, v in result["当前分位共振信息"].items():
            print(f"  {k}：{v}")
        print("-"*80)
        print(f"回测结果（{N1}年+{N2}年分位共振买入，持有1年）：")
        for quantile_label, res in result["回测结果（买入持有1年）"].items():
            print(f"\n{quantile_label}：")
            for k, v in res.items():
                print(f"  {k}：{v}")
        
        # 输出并保存交易明细
        print("\n" + "="*80)
        print("交易明细（前10行）：")
        trade_df = result["交易明细DataFrame"]
        if not trade_df.empty:
            print(trade_df.head(10))
            # 保存到Excel
            excel_filename = f"{fund_code}_{N1}年+{N2}年分位共振回测.xlsx"
            trade_df.to_excel(excel_filename, index=False)
            print(f"\n交易明细已保存至：{excel_filename}")
        else:
            print("暂无有效交易明细数据")

【调试】同时有近1年和近3年收益率的有效数据条数：3255
【调试】两个维度分位共振的买入候选数：719
【调试】各共振区间分布：
收益分位(80-100%)    202
收益分位(0-20%)      184
收益分位(20-40%)     152
收益分位(60-80%)      96
收益分位(40-60%)      85
Name: 1年分位标签, dtype: int64
基金代码：110020
有效数据总数：3255
共振买入候选数：719
--------------------------------------------------------------------------------
当前分位共振信息：
  当前近1年收益率(%)：22.03
  当前近1年收益率分位数(%)：79.17
  当前近3年收益率(%)：22.32
  当前近3年收益率分位数(%)：53.49
  当前近1年分位：收益分位(60-80%)
  当前近3年分位：收益分位(40-60%)
  当前是否分位共振：否
--------------------------------------------------------------------------------
回测结果（1年+3年分位共振买入，持有1年）：

收益分位(0-20%)：
  买入次数：184
  持有1年平均收益(%)：47.69
  持有1年收益0分位数：-0.87
  持有1年收益25分位数：14.61
  持有1年收益50分位数：28.48
  持有1年收益75分位数：79.86
  持有1年收益100分位数：139.52
  持有1年盈利概率(%)：99.46
  持有1年收益标准差(%)：41.57

收益分位(20-40%)：
  买入次数：152
  持有1年平均收益(%)：-0.57
  持有1年收益0分位数：-14.23
  持有1年收益25分位数：-9.93
  持有1年收益50分位数：-5.29
  持有1年收益75分位数：8.76
  持有1年收益100分位数：36.2
  持有1年盈利概率(%)：41.45
  持有1年收益标准差(%)：11.34

收益分位(80-100%)：
  买入次数：202
  持有1年平均收益(%)：-16.21
  持有

In [19]:
result["当前分位共振信息"]

{'当前近1年收益率(%)': 22.03,
 '当前近1年收益率分位数(%)': 79.17,
 '当前近3年收益率(%)': 22.32,
 '当前近3年收益率分位数(%)': 53.49,
 '当前近1年分位': '收益分位(60-80%)',
 '当前近3年分位': '收益分位(60-80%)',
 '当前是否分位共振': '是'}

In [10]:
result

{'基金代码': '110020',
 '近1年收益率(%)': 22.03,
 '历史分位数(%)': 81.86,
 '有效回测样本总数': 3755,
 '回测结果（买入持有1年）': {'收益分位(0-20%)': {'买入次数': 751,
   '持有1年平均收益(%)': 10.94,
   '持有1年盈利概率(%)': 59.52},
  '收益分位(20-40%)': {'买入次数': 751, '持有1年平均收益(%)': 12.0, '持有1年盈利概率(%)': 62.05},
  '收益分位(40-60%)': {'买入次数': 739, '持有1年平均收益(%)': 5.15, '持有1年盈利概率(%)': 48.44},
  '收益分位(60-80%)': {'买入次数': 613, '持有1年平均收益(%)': 8.71, '持有1年盈利概率(%)': 60.52},
  '收益分位(80-100%)': {'买入次数': 651, '持有1年平均收益(%)': -4.29, '持有1年盈利概率(%)': 37.48}}}